In [286]:
from src.smth2matrix.shift2zero import shift2zero
import numpy as np
from math import ceil, floor

def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(points1, points2):
    L1 = line(points1[0], points1[1])
    L2 = line(points2[0], points2[1])
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y
    else:
        return False

In [287]:
h = 1
points = np.array([[0.3, 3], [1, 0], [2.1, 0], [3, 3.7]])

size_of_sides = shift2zero(points)
n_x = ceil(size_of_sides[0] / h)
n_y = ceil(size_of_sides[1] / h)

lines = []
for _ in range(n_y+1):
    lines.append([])

for k in range(0, n_y+1):
    for i0 in range(points.shape[0]):
        if i0 == points.shape[0]-1:
            i1 = 0
        else:
            i1 = (i0 + 1)
        if i0 == 0:
            i2 = points.shape[0]-1
        else:
            i2 = (i0 - 1)
        #вершина лежит на линии
        if points[i0][1] == k*h:
            if points[i2][1] < k*h and points[i1][1] < k*h:
                print('hi', points[i0][0])
                lines[k].append([points[i0][0], points[i0][0]])
            elif points[i2][1] > k*h and points[i1][1] > k*h:
                print('ki',points[i0][0])
                lines[k].append([points[i0][0], points[i0][0]])
            else:
                print('bi',points[i0][0])
                lines[k].append([points[i0][0]])
        #ребро пересекает линию
        if points[i0][1] < k*h < points[i1][1] or points[i1][1] < k*h < points[i0][1]:
            print(i0, k)
            seg = intersection([points[i0], points[i1]], [[0, k*h], [n_x, k*h]])[0]
            lines[k].append(seg)
            print(intersection([points[i0], points[i1]], [[0, k*h], [n_x, k*h]])[0])


        #extension algorithm
        if k*(h + 1) > points[i0][1] > k*h >= points[i1][1] and points[i2][1] <= k*h:
            lines[k+1].append(points[i0][0])
            print(points[i0][0])
            lines[k+1].append(points[i0][0])
        elif points[i1][1] >= k*h > points[i0][1] > k*(h+1) and points[i2][1] >= k*h:
            lines[k+1].append(points[i0][0])
            print(points[i0][0])
            lines[k+1].append(points[i0][0])

print(lines)


bi 0.7
bi 1.8
0 1
0.46666666666666656
2 1
2.043243243243243
0 2
0.23333333333333328
2 2
2.2864864864864862
bi 0.0
2 3
2.5297297297297296
2.7
[[[0.7], [1.8]], [0.46666666666666656, 2.043243243243243], [0.23333333333333328, 2.2864864864864862], [[0.0], 2.5297297297297296], [2.7, 2.7]]


In [288]:
# %run C:\Users\79833\OneDrive\Документы\GitHub\NestingProblem-BMW2022\src\greedy_alg\class_pallets.py



from src.greedy_alg.class_pallets import Pallets
from src.class_packing import Packing
from src.smth2lines.polygon2segments import polygon2segments

h = 1
points = np.array([[0.3, 3], [1, 0], [2.1, 0], [3, 3.7]])

lines = polygon2segments(points, h)
print(lines)

packaging = Packing(width=200, height=30, drill_radius=0)

pallets = Pallets([20, 20])
pallets.add_pallet_lines()

print(pallets.pallet_lines)

[[0.7, 1.8], [0.46666666666666656, 2.043243243243243], [0.23333333333333328, 2.2864864864864862], [0.0, 2.5297297297297296], [2.7, 2.7]]
[[[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]], [[0, 0], [20, 20]]]


In [289]:
from src.class_item import Item

points = np.array([[0.3, 3], [1, 0], [2.1, 0], [3, 3.7]])
shift2zero(points)

[2.7, 3.7]

In [290]:
from copy import copy
segments = [
    [[0.7, 1.8], [2,3]], [[0.46666666666666656, 2.043243243243243]], [[0.23333333333333328, 2.2864864864864862]], [[0.0, 2.5297297297297296]], [[2.7, 2.7]]
]

segments_2 = [
    [[0.7, 1.8], [2,4]], [[0.46666666666666656, 2.043243243243243]], [[0.23333333333333328, 2.2864864864864862]], [[0.0, 2.5297297297297296]], [[2.7, 2.7]]
]

item_1 = Item(1, points)

item_1.segments = segments

item_2 = Item(2, points)

item_2.segments = segments_2

items = [copy(item_1), copy(item_2)]

for r in range(len(pallets.pallet_lines)):
    # print('change r')
    row = pallets.pallet_lines[r]
    for k in range(1, len(row)):
        for item in items:
            # print('change item')
            if not item.packed:
                t_vector = row[k-1][1] - item.segments[0][0][0]
                r_flag = 0
                ex_flag = 0
                i = 0
                l = len(item.segments[0])
                while i < len(item.segments) and ex_flag == 0:
                    # если первая строка объекта уже не помещается в отрезок
                    if i == 0 and item.segments[0][l-1][1] + t_vector > row[k][0]: # может быть [0][l-1][1][1]
                            ex_flag = 1
                    # если первая строка ещё помещается в отрезок
                    else:
                        j = 0
                        while j < len(item.segments[i]) and r_flag == 0:
                            m = 0
                            while r + i < len(pallets.pallet_lines) and m < len(pallets.pallet_lines[r+i]) and r_flag == 0:
                                # случаи пересечения линии и упаковки
                                if pallets.pallet_lines[r+i][m][0] <= item.segments[i][j][0] + t_vector < pallets.pallet_lines[r+i][m][1]:
                                    t_vector += pallets.pallet_lines[r+i][m][1] - (item.segments[i][j][0] + t_vector)
                                    i = 0
                                    r_flag = 1
                                    print('здесь1')
                                if pallets.pallet_lines[r+i][m][0] <= item.segments[i][j][1] + t_vector < pallets.pallet_lines[r+i][m][1]:
                                    t_vector += pallets.pallet_lines[r+i][m][1] - (item.segments[i][j][1] + t_vector)
                                    i = 0
                                    r_flag = 1
                                    print('здесь2')
                                # объект выходит за границы палеты
                                if item.segments[i][j][0] + t_vector < 0:
                                    t_vector = - (item.segments[i][j][0] + t_vector)
                                    i = 0
                                    r_flag = 1
                                    print('здесь3')
                                elif item.segments[i][j][1] + t_vector > pallets.shape[0]:
                                    ex_flag = 1
                                    r_flag = 1
                                    print('здесь4')
                                m += 1
                            j += 1
                    i+=1
                # если объект влезает, добавляем его на палету
                if ex_flag == 0:
                    # print('hello', r, 'id', item.id)
                    item.t_vector = t_vector
                    item.packed = True
                    # print('вектор: ',item.t_vector)
                    for s in range(len(item.segments)):
                        # print('сегмент: ',item.segments[s])
                        for l in range(len(item.segments[s])):
                            new_line = [item.segments[s][l][0] + item.t_vector, item.segments[s][l][1] + item.t_vector]
                            pallets.pallet_lines[r+s].append(copy(new_line))
                        pallets.pallet_lines[r+s].sort(key=lambda x: int(x[0]), reverse=False)
                        print('строка', r+s , ':', pallets.pallet_lines[r+s])





здесь3
строка 0 : [[0, 0], [0.9333333333333333, 2.033333333333333], [2.2333333333333334, 3.2333333333333334], [20, 20]]
строка 1 : [[0, 0], [0.7, 2.276576576576576], [20, 20]]
строка 2 : [[0, 0], [0.4666666666666667, 2.5198198198198196], [20, 20]]
строка 3 : [[0, 0], [0.2333333333333334, 2.763063063063063], [20, 20]]
строка 4 : [[0, 0], [2.9333333333333336, 2.9333333333333336], [20, 20]]
здесь1
строка 1 : [[0, 0], [0.7, 2.276576576576576], [2.7531531531531535, 3.853153153153153], [4.053153153153153, 6.053153153153153], [20, 20]]
строка 2 : [[0, 0], [0.4666666666666667, 2.5198198198198196], [2.51981981981982, 4.096396396396396], [20, 20]]
строка 3 : [[0, 0], [0.2333333333333334, 2.763063063063063], [2.2864864864864867, 4.339639639639639], [20, 20]]
строка 4 : [[0, 0], [2.9333333333333336, 2.9333333333333336], [2.0531531531531533, 4.582882882882883], [20, 20]]
строка 5 : [[0, 0], [4.7531531531531535, 4.7531531531531535], [20, 20]]
